In [3]:
import pyprob
from pyprob.distributions import Normal
import pyprob.diagnostics
import torch
import pandas
import os

def parse_n_infect_from_output(file_name):
    data = pandas.read_csv(file_name, sep='\t', names = ["survey_number", "third_dimension", "measure", "value"])
    n_infect = torch.from_numpy(data['value'][:12].values)
    return n_infect

PATH = '/home/examples/matsari_intervention/'
def forward_before():
    exists = os.path.isfile(PATH + 'output.txt')
    if exists:
        print('Removing output...')
        os.remove(PATH+'output.txt')
    
def forward_after():
    n_infect = parse_n_infect_from_output(PATH + 'output.txt')
    likelihood = Normal(n_infect, 0.01)
    pyprob.observe(likelihood, name='n_infect')

# This will save in /code/notebooks
file_name = 'trace'
address_dict_file_name = 'address_dict'
use_address_base=True
model = pyprob.RemoteModel('ipc://@openmalaria_probprog', before_forward_func=forward_before ,after_forward_func=forward_after, address_dict_file_name=address_dict_file_name)

In [2]:
trace = next(model._trace_generator())

ppx (Python): zmq.REQ socket connecting to server ipc://@openmalaria_probprog
ppx (Python): This system        : pyprob 0.13.2.dev18
ppx (Python): Connected to system: pyprob_cpp 0.1.9 (master:7290be2)
ppx (Python): Model name         : OpenMalaria probprog


FileNotFoundError: [Errno 2] File b'/home/examples/test/output.txt' does not exist: b'/home/examples/test/output.txt'

## Generating the Prior

In [ ]:
prior = model.prior_traces(num_traces=5)

Time spent  | Time remain.| Progress             | Trace | Traces/sec
ppx (Python): zmq.REQ socket connecting to server ipc://@openmalaria_probprog
ppx (Python): This system        : pyprob 0.13.2.dev18
ppx (Python): Connected to system: pyprob_cpp 0.1.9 (master:7290be2)
ppx (Python): Model name         : OpenMalaria probprog
Removing output...


## Analysing the Prior

In [ ]:
# print(len(prior))
# trace=prior._values[0]
prior._values
trace=prior._values[1]
var = trace.variables
# len(var)
# print(var)
count  = 1
for k in trace.variables_dict_address_base:
    print(" Address {2} : {0} \n Value : {1} \n\n".format(k,trace.variables_dict_address_base[k],count))
    count += 1
#pyprob.diagnostics.graph(prior, file_name="plots/mpenny_adapted_17")
#print(prior.save_metadata)

## Generating to the posterior by conditioning on the number of infected individuals for each demographic

In [ ]:
 posterior_dist = model.posterior_traces(num_traces=10, inference_engine=pyprob.InferenceEngine.RANDOM_WALK_METROPOLIS_HASTINGS, observe={'n_infect': parse_n_infect_from_output(PATH+'output.txt')})

In [ ]:
print(posterior_dist.values_numpy())

### Inference with Importance sampling

In [ ]:
posterior_dist_IS = model.posterior_traces(num_traces=10, inference_engine=pyprob.InferenceEngine.IMPORTANCE_SAMPLING, observe={'n_infect': parse_n_infect_from_output(PATH+'output.txt')})

#### View observations

In [ ]:
# ground_truth_trace = next(model._trace_generator())
prior.named_variables['n_infect'].value

In [ ]:
is_posterior_traces = model.posterior_traces(observe={'n_infect': ground_truth_prevalence}, num_traces=5)
is_posterior_population = is_posterior_traces.map(lambda trace: trace.named_variables['population'].value)
is_posterior_prevalence = is_posterior_population.map(get_prevalence)

## Plot Histogram of Addresses

In [ ]:
# we currently do not have a ground truth for this scenario run with a small pop size
pyprob.diagnostics.address_histograms(dists, plot=True, plot_show=True, file_name=file_name, ground_truth_trace=ground_truth_trace, use_address_base=use_address_base)